In [ ]:
import requests
from requests.exceptions import TooManyRedirects
from bs4 import BeautifulSoup
import os
import pandas as pd
from tqdm import tqdm
import time
import re

# Create a folder downloads to store all the software to be downloaded. 
download_suffix = "download"

# Headers to be sent with requests to mimic a real browser
headers = {
    'Referer': 'https://software.informer.com/software/',
    'Sec-Ch-Ua': '"Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"',
    'Sec-Ch-Ua-Mobile': '?0',
    'Sec-Ch-Ua-Platform': '"macOS"',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'
}

# Create the downloads directory if it does not exist
if not os.path.exists('downloads'):
    os.makedirs('downloads')


def get_soup(url):
    """Fetch the content from the URL and return a BeautifulSoup object."""
    try:
        response = requests.get(url, headers=headers, timeout=10, allow_redirects=True)
        response.raise_for_status()
        return BeautifulSoup(response.content, 'html.parser')
    except TooManyRedirects:
        print(f"Too many redirects for URL: {url}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Request failed for URL: {url}, Error: {e}")
        return None


def scrape_application_page(app_url):
    """Scrape application details from the application page."""
    app_soup = get_soup(app_url + download_suffix)
    if app_soup is None:
        return None
    
    try:
        download_link_tag = app_soup.find('a', class_='download_btn')
        download_link = download_link_tag['href'] if download_link_tag else None
        if not download_link or download_link.startswith('#'):
            print(f"Invalid download link for {app_url}. Skipping...")
            return None

        print(f"Downloaded metadata for {app_url}")
        return download_link

    except Exception as e:
        print(f"Error scraping {app_url}: {e}")
        return None


def sanitize_filename(filename):
    """Sanitize the filename by removing unwanted characters."""
    return re.sub(r'[<>:"/\\|?*]', '_', filename)


def download_file(url, file_name):
    """Download the file from the given URL and save it to the 'downloads' directory."""
    try:
        response = requests.get(url, headers=headers, stream=True)
        response.raise_for_status()
        
        file_name = sanitize_filename(file_name)

        with open(f'downloads/{file_name}', 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
                    
    except requests.exceptions.TooManyRedirects:
        print(f"Too many redirects for URL: {url}. Skipping download...")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading {url}: {e}")


# ==============================
# Load file_info.csv để lấy danh sách title hợp lệ
# ==============================
file_df = pd.read_csv("/kaggle/input/file-info-benign/file_info.csv")

valid_titles = set(
    f.replace("report_", "").replace(".exe.json", "").strip()
    for f in file_df["filename"]
)

print(f"Loaded {len(valid_titles)} valid titles from file_info.csv")


def scrape_page(page_number, downloaded_files):
    """Scrape all applications on a given page."""
    url = f"https://software.informer.com/software/{page_number}/"
    soup = get_soup(url)
    if soup is None:
        print(f"Error getting soup for page {page_number}. Skipping...")
        return downloaded_files
    
    apps = soup.find_all('a', class_='link_ttl')
    
    for app in apps:
        if downloaded_files >= 500:
            print("Downloaded 500 files. Stopping the scraping process.")
            return downloaded_files
        
        app_href = app['href']
        app_title = app.text.strip()

        # chỉ download nếu nằm trong danh sách hợp lệ
        if app_title not in valid_titles:
            print(f"Skip {app_title} (not in file_info list)")
            continue
        
        app_description_tag = app.find_next('div', class_='description')
        if not app_description_tag:
            app_description_tag = app.find_next('p')
        app_description = app_description_tag.text.strip() if app_description_tag else "No description available"
        
        download_link = scrape_application_page(app_href)

        if download_link:
            file_name = f"{sanitize_filename(app_title)}.exe"  # Assuming .exe format
            download_file(download_link, file_name)
            application_data.append({
                "title": app_title,
                "description": app_description,
                "href": app_href,
                "Download link": download_link
            })
            downloaded_files += 1
    
    return downloaded_files


application_data = []
downloaded_files = 0

# Scrape pages
for page in tqdm(range(1, 300), desc="Scraping pages"): 
    if downloaded_files == 137:
        break
    downloaded_files = scrape_page(page, downloaded_files)
    time.sleep(1)

# Save results
df = pd.DataFrame(application_data)
df.to_csv('application_data.csv', index=False)

print("Scraping completed and data saved to application_data.csv")
print(f"Scraping completed and {downloaded_files} software saved in downloads folder.")

Loaded 138 valid titles from file_info.csv


Scraping pages:   0%|          | 0/299 [00:00<?, ?it/s]

Skip VLC media player (not in file_info list)
Skip µTorrent (uTorrent) (not in file_info list)
Skip TeamViewer (not in file_info list)
Skip 7-Zip (not in file_info list)
Skip The KMPlayer (not in file_info list)
Skip Dropbox (not in file_info list)
Skip Notepad++ (not in file_info list)
Skip PhotoScape (not in file_info list)
Skip Free Download Manager (not in file_info list)
Skip WhatsApp (not in file_info list)
Skip Norton Security (not in file_info list)
Skip ESET Smart Security (not in file_info list)
Downloaded metadata for https://libreoffice.informer.com/
Skip IrfanView (not in file_info list)
Skip FastStone Image Viewer (not in file_info list)
Skip foobar2000 (not in file_info list)
Skip PDF Reader (not in file_info list)
Skip KeePass Password Safe (not in file_info list)
Skip Wise Registry Cleaner Free (not in file_info list)
Skip ISO to USB (not in file_info list)
Skip Slack (not in file_info list)
Skip Wondershare MirrorGo (not in file_info list)
Skip Telegram Desktop (not i

Scraping pages:   0%|          | 1/299 [03:28<17:15:43, 208.54s/it]

Skip NVIDIA Graphics Driver (not in file_info list)
Skip QuickTime (not in file_info list)
Skip Steam (not in file_info list)
Skip GOM Player (not in file_info list)
Skip K-Lite Codec Pack (not in file_info list)
Skip Yahoo! Messenger (not in file_info list)
Skip Foxit Reader (not in file_info list)
Skip CyberLink YouCam (not in file_info list)
Skip FormatFactory (not in file_info list)
Skip AMD Catalyst (not in file_info list)
Skip WinZip (not in file_info list)
Skip Dropbox (not in file_info list)
Skip Unity Web Player (not in file_info list)
Skip AIMP (not in file_info list)
Skip Google Drive (not in file_info list)
Skip iCloud (not in file_info list)


...

Skip EasyBits GO (not in file_info list)
Skip ActionVoip (not in file_info list)
Skip Dolby Home Theater (not in file_info list)
Skip Chronicles of Albian (not in file_info list)
Skip FxPro - MetaTrader (not in file_info list)
Skip FirstClass (not in file_info list)
Skip 18 Wheels of Steel Pedal (not in file_info list)
Skip Lexmark Fax Solutions (not in file_info list)
Skip Who's Your Daddy (not in file_info list)
Skip GTA San Andreas Multiplayer (not in file_info list)
Skip Google Calendar Sync (not in file_info list)
Skip Scratch LIVE (not in file_info list)
Skip Invention 2 (not in file_info list)
Skip Moffsoft FreeCalc (not in file_info list)
Skip VoxOx (not in file_info list)
Skip SavvyConnect (not in file_info list)


Scraping pages: 100%|█████████▉| 298/299 [42:13<00:01,  1.27s/it]

Skip ScpToolkit (not in file_info list)
Skip FireAlpaca (not in file_info list)
Skip SecEditCtl.BOC (not in file_info list)
Skip Free PDF Compressor (not in file_info list)
Skip RecoveRx (not in file_info list)
Skip One Touch Upgrade S (not in file_info list)
Skip ONOFF Charge B15.0709.1 (not in file_info list)
Skip Free FreeCell Solitaire (not in file_info list)
Skip This War of Mine (not in file_info list)
Skip Spyware Clear (not in file_info list)
Skip Dino Storm (not in file_info list)
Skip Remote PC Server (not in file_info list)
Skip Nexus Root Toolkit (not in file_info list)
Skip EcoSmartFilter (not in file_info list)
Skip CyberLink Power2Go Content (not in file_info list)
Skip FileSearchy Pro (not in file_info list)


Scraping pages: 100%|██████████| 299/299 [42:14<00:00,  8.48s/it]

Scraping completed and data saved to application_data.csv
Scraping completed and 107 software saved in downloads folder.
